In [1]:
import glob
import pandas as pd
import time
import datetime as dt
import os
import numpy as np
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pathlib
from pathlib import Path
import pickle
import glob
from datetime import datetime
warnings.filterwarnings("ignore")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
import math
import plotly.express as px
import sys
from scipy.stats import skew, kurtosis
from scipy.stats import spearmanr
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.metrics import mean_tweedie_deviance
import sys
import umap.umap_ as umap
def mean_absolute_percentage_error(y_true, y_predicted):
    y_true, y_predicted = np.array(y_true), np.array(y_predicted)
    return np.mean(np.abs((y_true - y_predicted) / y_true)) * 100
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
os.chdir('/Users/gowtham/Downloads/Final_200/200_mesh_final/Raw_Data')

In [2]:
Agg_data=pd.read_csv("/Users/gowtham/Downloads/Final_200/200_mesh_final/Raw_Data/Precip_Raw_Data_200.csv")

In [5]:
from utils.constants import final_cols,columns_to_aggregate,columns_to_resample,resample_frequency,\
agg_functions,min_shifts,max_shifts,shifts_per_day,IST_TIMEZONE,Next_Predictions,best_cols,best_model
from data_preprocessing import replace_outliers_with_median,initial_data_preprocessing,\
data_aggregation,data_filter,calculate_lag_dates,preparing_lag_data
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pickle
from utils.common import get_current_datetime,sendmail

ModuleNotFoundError: No module named 'utils'

In [ ]:
def main_prediction(final_cols,best_cols,best_model):
    df_unseen_PSD = None
    if Next_Predictions is True:
        final_Agg_data=Agg_data[final_cols]
        treated_agg_data = replace_outliers_with_median(final_Agg_data)
        df_1hr=initial_data_preprocessing(treated_agg_data)
        resampled_df_8hr=data_aggregation(df_1hr)
        filtered_df=data_filter(resampled_df_8hr)
        final_df=filtered_df.copy()
        max_timestamp = final_df['Timestamp_To'].max()
        max_timestamp_dt = pd.to_datetime(max_timestamp)
        future_timestamp = max_timestamp_dt + timedelta(days=13)
        target_date_str = future_timestamp.strftime("%Y-%m-%d %H:%M")
        start_date_min_lag, start_date_max_lag = calculate_lag_dates(target_date_str, min_shifts, max_shifts, shifts_per_day)
        print("Start Date for Minimum Lag:", start_date_min_lag.strftime("%Y-%m-%d %H:%M"))
        print("Start Date for Maximum Lag:", start_date_max_lag.strftime("%Y-%m-%d %H:%M"))
        final_df_lag = final_df[(final_df['Timestamp_To'] <= start_date_min_lag) & (final_df['Timestamp_To'] >= start_date_max_lag)]
        if len(final_df_lag)<45:
            expected_timestamps = pd.date_range(start=start_date_max_lag, end=start_date_min_lag, freq='8H')
            missing_timestamps = expected_timestamps.difference(final_df_lag['Timestamp_To'])
            missing_rows = pd.DataFrame({col: np.nan for col in final_df_lag.columns}, index=missing_timestamps)
            missing_rows['Timestamp_To'] = missing_timestamps
            final_df_lag = pd.concat([final_df_lag, missing_rows], ignore_index=True)
            final_df_lag.sort_values(by='Timestamp_To', inplace=True)
        final_df_lag.sort_values(by='Timestamp_To', inplace=True)
        final_df_lag = final_df_lag.interpolate(method='linear',
                                               limit_direction='forward')
        final_df_lag = final_df_lag.interpolate(method='linear',
                                               limit_direction='backward')
        final_df_lag.set_index('Timestamp_To', inplace=True)
        df_lag_cleaned=preparing_lag_data(final_df_lag)
        with open(best_cols, 'rb') as file:
            final_cols = pickle.load(file)
        df_final_cleaned=df_lag_cleaned[final_cols]
        target_column ='Feed_Hydrate_D50'
        X_unseen = df_final_cleaned.drop([target_column],axis=1).copy()
        y_unseen = df_final_cleaned[[target_column]].copy()
        with open(best_model, 'rb') as file:
            model = pickle.load(file)
        preds = model.predict(X_unseen.values)
        print(f"we are predicting for {future_timestamp}")
        preds=preds.round(2)
        df_unseen_PSD=pd.DataFrame()
        df_unseen_PSD['Timestamp_To']=X_unseen.index+ timedelta(days=13)
        df_unseen_PSD['PSD_Prediction']=preds.round(2)
        return df_unseen_PSD
    else:
        print("Latest shift data is not present")
        
        return df_unseen_PSD
        